# Smollmv2


## Install Depedencies

In [1]:
!pip install lightning --quiet
!pip install tiktoken --quiet
!pip install datasets --quiet
!pip install transformers --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 927.3/927.3 kB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 819.3/819.3 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requ

## Original Model [Smollmv2-135M]


In [2]:
# Import Tokenizer and LLM
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the model
checkpoint = "HuggingFaceTB/SmolLM2-135M"
device = "cuda" # for GPU usage or "cpu" for CPU usage
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

# Print the model
print(model)

# Print total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params:,}\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(49152, 576)
    (layers): ModuleList(
      (0-29): 30 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=192, bias=False)
          (v_proj): Linear(in_features=576, out_features=192, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((576,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((576,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm

---

## Reverse Engineered Model

The key changes made to the Decoder only model are:

- Removed position embeddings in favor of RoPE (Rotary Position Embeddings)

- Split the attention into separate Q,K,V projections with reduced dimensions for K,V

- Kept the MLP structure with gate/up/down projections

These changes align your model architecture more closely with modern transformer designs like Llama, while maintaining efficiency through reduced KV dimensions

In [3]:
# Clone the project repository and add it to the system path
!git clone https://github.com/Shilpaj1994/SmoLLMv2.git

# Add it to the path
import sys
sys.path.insert(0,'./SmoLLMv2/')

Cloning into 'SmoLLMv2'...
remote: Enumerating objects: 30, done.
remote: Counting objects: 100% (30/30), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 30 (delta 10), reused 30 (delta 10), pack-reused 0 (from 0)
Receiving objects: 100% (30/30), 15.84 KiB | 5.28 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [4]:
# Load the config and model
from config import SmollmConfig, RoPEConfig
from smollmv2 import SmollmV2

# Print the model
model = SmollmV2(SmollmConfig())
print(model)

# Print total number of parameters in the model
total_params = sum(p.numel() for p in model.parameters())
print(f"Total model parameters: {total_params:,}\n")

SmollmV2(
  (transformer): ModuleDict(
    (wte): Embedding(49152, 576)
    (h): ModuleList(
      (0-29): 30 x Block(
        (ln_1): LayerNorm((576,), eps=1e-05, elementwise_affine=True)
        (attn): CausalSelfAttention(
          (q_proj): Linear(in_features=576, out_features=576, bias=False)
          (k_proj): Linear(in_features=576, out_features=189, bias=False)
          (v_proj): Linear(in_features=576, out_features=189, bias=False)
          (o_proj): Linear(in_features=576, out_features=576, bias=False)
        )
        (ln_2): LayerNorm((576,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (gate_proj): Linear(in_features=576, out_features=1536, bias=False)
          (up_proj): Linear(in_features=576, out_features=1536, bias=False)
          (down_proj): Linear(in_features=1536, out_features=576, bias=False)
        )
      )
    )
    (ln_f): LayerNorm((576,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=576, out_features=49

---

## Model Training

Trained the model on [Smollm-corpus](https://huggingface.co/datasets/HuggingFaceTB/smollm-corpus/tree/main/cosmopedia-v2) dataset for 5000 steps using streaming functionality for the dataset rather than downloading it

In [5]:
# Import the main training function
from smollv2_lightning import main

# Train fro 5000 steps
main(interupt_steps=5000)

Train by epochs or steps? (e/s): s
Enter number of steps: 5000

No checkpoints found. Starting fresh training...


tokenizer_config.json:   0%|          | 0.00/3.91k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/489 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loops/optimization/automatic.py:134: `training_step` returned `None`. If this was on purpose, ignore this warning...


Validation: |          | 0/? [00:00<?, ?it/s]


Validation - loss: 6.4842 | dt: 104.82ms
GPU Memory: 2.04GB / 5.00GB


INFO:pytorch_lightning.profilers.profiler:FIT Profiler Report

----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Action                                                                                                                                                                   	|  Mean duration (s)	|  Num calls      	|  Total time (s) 	|  Percentage %   	|
----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|  Total                                                                                                                              


Generating learning rate plot...
Learning rate plot saved as 'learning_rate_schedule.png'


---

## Training from last checkpoint

Train for another 50 steps from the last checkpoint

In [6]:
# Import the main training function
from smollv2_lightning import main

# Train fro 50 steps
main(interupt_steps=50)

Train by epochs or steps? (e/s): s
Enter number of steps: 50

Found existing checkpoint: checkpoints/last.ckpt
Resume training from checkpoint? (y/n): y

Resuming training from checkpoint: checkpoints/last.ckpt


/content/./SmoLLMv2/smollv2_lightning.py:383: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)


Compiling model for faster training...
Total model parameters: 134,411,328



INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



Starting training with performance monitoring...
Format: step | loss | iteration time | tokens per second | GPU memory



Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/104 [00:00<?, ?it/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /content/checkpoints exists and is not empty.
INFO:pytorch_lightning.utilities.rank_zero:Restoring states from the checkpoint path at checkpoints/last.ckpt
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type            | Params | Mode 
--------------------------------------------------
0 | model | OptimizedModule | 134 M  | train
--------------------------------------------------
134 M     Trainable params
0         Non-trainable params
134 M     Total params
537.645   Total estimated model params size (MB)
367       Modules in train mode
0         Modules in eval mode
/usr/local/lib/python3.11/dist-packages/pytorch_lightning/loggers/tensorboard.py:195: Could not log computational graph to TensorBoard: The `model.example_input_array` attribute is not set or `input_array` was not 


Generating learning rate plot...
Learning rate plot saved as 'learning_rate_schedule.png'


---

## Inference

In [7]:
import torch
from transformers import GPT2Tokenizer
from smollv2_lightning import LitSmollmv2
from cosmopedia_datamodule import CosmopediaDataModule
from config import SmollmConfig, DataConfig
from inference import generate_text, load_model


# Path to your checkpoint
checkpoint_path = "./checkpoints/last.ckpt"

# Load the model
model = load_model(checkpoint_path)
print("Model loaded successfully!")

# Example prompts for generation
prompts = [
    "Once upon a time",
    "The future of artificial intelligence",
    "In the distant galaxy"
]

# Generate text for each prompt
for prompt in prompts:
    print("\nPrompt:", prompt)
    generated = generate_text(prompt=prompt, model=model)
    print("Generated:", generated)
    print("-" * 50)

Compiling model for faster training...
Total model parameters: 134,411,328

Model loaded successfully!

Prompt: Once upon a time
Generated: Once upon a time, there was a small town named the phase of a curious land namedian named "Can you know a place," which tells us all the big Atlantic of her favorite white, and even a family of the fascinating world. This way, the seem like how much we have the world around theSo we'll learn more about ourselves and learn new ways to understand and the history of the world.

One day, a young helping us learn about the world around us. It's a type of disorder
--------------------------------------------------

Prompt: The future of artificial intelligence
Generated: The future of artificial intelligence in the late 10th century. This book has been tiny because the late 180s, the side of the 200th century, an foreshad century for 170 years of the World War 180s of the ' \N. You might the story of the clothing; they became known for the summary of the